In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## Coletando nossos dados

In [ ]:
url = 'https://raw.githubusercontent.com/Mirlaa/restart_dg_2ed_aula/main/Dados/dados_restart.csv'

In [ ]:
df = pd.read_csv(url)
df.head()

## Do que se tratam os dados

**Dicionário de dados:** Informação sobre nossos dados.

- `id`: Identificação única de cada registro;
- `idade`: Idade do cliente;
- `profissao`: Profissão do cliente;
- `estado_civil`: Estado civil do cliente;
- `educacao`: Nível de educação do cliente;
- `inadimplencia`: Indica se o cliente está inadimplente
- `saldo`: Saldo na conta do cliente;
- `habitacao`: Indica se o cliente possui habitação;
- `emprestimo`: Indica se o cliente possui empréstimo;
- `contato`: Tipo de contato com o cliente;
- `dia`: Dia do mês do último contato;
- `mes`: Mês do último contato;
- `duracao`: Duração da última ligação em segundos;
- `campanha`: Número de contatos realizados durante esta campanha;
- `contatos_anteriores`: Número de contatos realizados antes desta campanha;
- `resultado_campanha_anterior`: Resultado da campanha anterior; e
- `alvo`: Alvo da campanha.

In [ ]:
df.info()

**Observações:**

- existem várias colunas com valores nulos
- todos os valores estão no tipo apropriado, caso não estivessem usaríamos o método [`astype`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html)

In [ ]:
df.describe()

In [ ]:
df.describe(include="object")

**Observações:**

- Coluna `idade` com valores irreais
- Valores nulos em algumas colunas

In [ ]:
df.nunique()

**Observações:**

- Coluna `id` mostra que tem ids duplicados
- Temos colunas numéricas com apenas duas categorias, podem ser binários

## Lidando com duplicados e nulos

## Dados duplicados

Amostras duplicadas são dados que possuem **os mesmos valores** em colunas correspondentes.

Por exemplo, não há distinção da amostra A para amostra B, justamente porque todos os valores de colunas correspondentes são iguais. Por isso, transmitem **apenas uma informação.**

In [ ]:
df.duplicated()

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated()]

O que fazer com essas amostras? Devemos removê-las do banco de dados.

Podemos citar três motivos principais para a remoção das amostras duplicadas:

- **Viés do modelo**: se há amostras duplicadas no conjunto de dados, pode ser que o modelo de machine learning dê mais importância para essas amostras repetidas.
- **Melhora do desempenho do modelo**: se inserimos amostras duplicadas, vão ser necessários mais cálculos e poder de processamento, além de ser um desperdício computacional trabalhar com amostras com o mesmo valor e que transmitem a mesma informação.
- **Aumento da qualidade dos resultados**: vamos inserir informações únicas, sem dados repetidos. Ou seja, vão ser mais relevantes para o modelo.

In [ ]:
df.drop_duplicates(inplace=True)
df

In [ ]:
df.duplicated().sum()

## Dados nulos

Dados nulos são valores que estão ausentes ou são desconhecidos na base de dados.

In [ ]:
df.isnull()

In [ ]:
df.isnull().sum()

**Observações:**

- Apenas as colunas `idade` e `saldo` são colunas numéricas e ambas contém pouquíssimos dados nulos
- As colunas `profissao`, `educacao`, `resultado_campanha_anterior` contém dados nulos com diferentes valores.

### Dados numéricos

Não sabemos os valores originais em `idade` e `saldo`, mas não podemos substituir por qualquer valor.

- Viés nos dados: o modelo de machine learning vai tentar procurar padrões que vão estar incorretos, pois não são os dados reais. Isso pode levar a previsões enganosas, incorretas e que não são satisfatórias.
- Distorção de resultados: inserir um valor que não é o correto faz com que o modelo aprenda com dados incorretos e, consequentemente, levam a previsões que podem ser incorretas.

In [ ]:
df.dropna(subset=['idade','saldo'])

In [ ]:
df.dropna(subset=['idade','saldo'], inplace=True)

In [ ]:
df.isnull().sum()

### Dados textuais

In [ ]:
for col in ['profissao','educacao','resultado_campanha_anterior']:
    print(f"Coluna: {col}")
    print(df[col].unique())
    print("\n")

In [ ]:
df['profissao'].fillna('desconhecido', inplace = True)
df['educacao'].fillna('desconhecido', inplace = True)
df['resultado_campanha_anterior'].fillna('desconhecido', inplace = True)

In [ ]:
df.isnull().sum()

Vamos resetar os índices devido a esses índices saltados que devem ser evitados em machine learning.

In [ ]:
df.reset_index(drop=True, inplace=True)

## Verificando outliers

Outliers, que são valores atípicos ou um ponto fora da curva. Em estatística, isso significa um dado que se distancia muito da distribuição padrão que os dados apresentam.

In [ ]:
df_numericos = df.select_dtypes(include="number")
df_numericos.head()

**Conhecendo o boxplot:**

![](https://github.com/Mirlaa/restart_dg_2ed_aula/blob/main/boxplot.png?raw=true)

No centro da caixa, encontramos a mediana, que divide os dados ao meio, com 50% à esquerda e 50% à direita.

Nos cantos superiores esquerdo e direito da caixa, estão os quartis Q1 e Q3, que dividem os dados em quatro partes iguais, cada uma com 25% dos dados. Até Q1, temos os 25% menores dados, enquanto Q3 abrange os 25% maiores.

Entre Q1 e Q3, encontramos os 75% dos dados, onde Q2 é a mediana. A partir de Q3, temos os 25% finais dos dados.

Acima da caixa, calculamos o IIQ (Intervalo Interquartil), que é a diferença entre Q3 e Q1.

À esquerda e à direita da caixa, existem duas arestas. Na aresta esquerda, calculamos o limite inferior como "Q1 - 1,5 x IIQ", e na aresta direita, o limite superior como "Q3 + 1,5 x IIQ". Valores abaixo do limite inferior ou acima do limite superior são considerados candidatos a outliers, que estamos buscando identificar.


```Python
import pandas as pd

# Calcular os quartis e a amplitude interquartil (IIQR) para a coluna específica
Q1 = df['nome_da_coluna'].quantile(0.25)
Q3 = df['nome_da_coluna'].quantile(0.75)
IIQR = Q3 - Q1

# Filtrar o DataFrame para remover outliers com base na coluna específica
df_sem_outliers = df[~((df['nome_da_coluna'] < (Q1 - 1.5 * IIQR)) | (df['nome_da_coluna'] > (Q3 + 1.5 * IIQR)))]
```

In [ ]:
df_numericos.describe()

In [ ]:
plt.rcParams["figure.figsize"] = (22, 3)
sns.boxplot(x=df['idade'])

In [ ]:
df = df[df['idade']>= 18]
df = df[df['idade']< 120]

In [ ]:
plt.rcParams["figure.figsize"] = (22, 3)
sns.boxplot(x=df['idade'])

In [ ]:
sns.boxplot(x=df['contatos_anteriores'])

In [ ]:
df = df[df['contatos_anteriores']<=40]

In [ ]:
sns.boxplot(x=df['contatos_anteriores'])

In [ ]:
df = df.reset_index(drop=True)

# Trabalhando com dados categóricos

## Ajuste de dados de texto

In [ ]:
df

In [ ]:
df_textos = df.select_dtypes(exclude="number")
df_textos.head()

**Observações:**

- Quando vemos a coluna `contato` notamos que existem dados **vazios** nas primeiras linhas, vamos verificar que tipo de dado é esse.

In [ ]:
for col in df_textos.columns:
    print(f"Coluna: {col}")
    print(df_textos[col].unique())
    print("\n")

In [ ]:
df_textos.nunique()

In [ ]:
df['contato'] = df['contato'].replace(' ', 'outro')

In [ ]:
df['contato'].unique()

Variáveis categóricas são uma forma de agrupar informações em categorias diferentes, sem um valor numérico associado a elas, como o caso das variáveis numéricas.

Não conseguimos atribuir um valor numérico para essas variáveis, apenas classificá-las em categorias distintas.

O ***One-Hot Encoding*** é uma técnica de pré-processamento usada para lidar com colunas categóricas em um DataFrame, transformando essas categorias em representações binárias, onde cada categoria se torna uma nova coluna binária (0 ou 1).

Vamos ver um exemplo, temos um DataFrame que possui uma coluna de cores com valores categóricos:

|    | Cor      |
|---:|:---------|
|  0 | Vermelho |
|  1 | Azul     |
|  2 | Verde    |
|  3 | Vermelho |
|  4 | Amarelo  |

Ao aplicar o *One-Hot Encoding* convertemos cada cor (categoria) em uma coluna binária:

| | Cor_Amarelo | Cor_Azul | Cor_Verde | Cor_Vermelho |
|---:|-----:|---------:|-------:|-------:|
|  0 |  0 |          0 |0 |   1 |
|  1 |  0 |          1 |0 |   0 |
|  2 |  0 |          0 |1 |   0 |
|  3 |  0 |          0 |0 |   1 |
|  4 |  1 |          0 |0 |   0 |

Cada categoria única na coluna 'Cor' se torna uma nova coluna binária no DataFrame codificado. Por exemplo, 'Vermelho' se torna uma coluna chamada 'Cor_Vermelho', onde temos 1 para as linhas em que a cor é 'Vermelho' e 0 caso contrário. O mesmo princípio se aplica às outras cores.

- Existe a aplicação do *One-Hot Encoding* com Pandas utilizando [`get_dummies`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html).

```Python
df = pd.get_dummies(df)

```

Mas podemos fazer diferente.

In [ ]:
df.drop(['profissao'], axis=1, inplace=True)

In [ ]:
categorias = {
    'estado_civil': ['casado(a)', 'solteiro(a)', 'divorciado(a)'],
    'educacao': ['secundário', 'terciário', 'desconhecido', 'primário'],
    'inadimplencia': ['sim'],
    'habitacao': ['sim'],
    'emprestimo': ['sim'],
    'contato': ['outro', 'celular', 'telefone'],
    'resultado_campanha_anterior': ['desconhecido', 'falha', 'outro', 'sucesso'],
}

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import joblib

codificador = OneHotEncoder(categories=[categorias[col] for col in categorias], sparse_output=False, handle_unknown='ignore')

colunas_codificadas = codificador.fit_transform(df[list(categorias.keys())])

In [ ]:
colunas_codificadas

Podemos salvar o encoder para aplicar em outro conjunto de dados, como o conjunto de teste.

In [ ]:
joblib.dump(codificador, 'modelo_onehotencoder.pkl')

Para carregar o modelo posteriormente

```Python
import pandas as pd
import joblib

encoder_loaded = joblib.load('modelo_onehotencoder.pkl')

# novos_dados é um outro conjunto de dados

colunas_categoricas = ['profissao', 'estado_civil', 'educacao', 'inadimplencia', 'habitacao',
                       'emprestimo', 'contato', 'mes', 'resultado_campanha_anterior']

dados_codificados_novos = codificador_carregado.transform(novos_dados[colunas_categoricas])
```

In [ ]:
df_codificado = pd.DataFrame(colunas_codificadas, columns=codificador.get_feature_names_out())

In [ ]:
df_codificado

In [ ]:
df_final = pd.concat([df, df_codificado], axis=1)

df_final.drop(columns=list(categorias.keys()), inplace=True)

In [ ]:
df_final

### Dados da coluna alvo

**Observações:**

- Ainda existe uma coluna em formato de texto: `alvo`

In [ ]:
binario_map = {
    'sim': 1,
    'não': 0
}

df_final['alvo'].map(binario_map)

In [ ]:
df_final['alvo'] = df_final['alvo'].map(binario_map)

In [ ]:
mes_map = {
    'janeiro': 1, 'fevereiro': 2, 'março': 3, 'abril': 4, 'maio': 5, 'junho': 6,
    'julho': 7, 'agosto': 8, 'setembro': 9, 'outubro': 10, 'novembro': 11, 'dezembro': 12
}

df_final['mes'].map(mes_map)

In [ ]:
df_final['mes'] = df_final['mes'].map(mes_map)

In [ ]:
df_final

In [ ]:
df_final.to_csv('dados_restart_tratados.csv', index=False)

# Visualizações

In [ ]:
df_final.columns

In [ ]:
correlacoes = np.corrcoef(df_final.values, rowvar=False)
correlacoes

- É possível também aplicar [`df.corr`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) para ter a matriz de correlação.

Podemos buscar observar em um gráfico de calor os valores de correlação.

In [ ]:
correlacoes_df = pd.DataFrame(correlacoes, index=df_final.columns, columns=df_final.columns)

plt.figure(figsize=(10, 8))
sns.heatmap(correlacoes_df, cmap='coolwarm', vmin=-1, vmax=1, annot = False)
plt.title('Matriz de Correlação')
plt.show()

Temos muitas colunas para deixar na matriz.

In [ ]:
correlacao_alvo = correlacoes_df['alvo'].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(10, 8))
sns.barplot(x=correlacao_alvo.values, y=correlacao_alvo.index, palette='coolwarm')
plt.title('Correlação com a Variável Alvo')
plt.show()

# Documentação usada no projeto

Links da documentação para os métodos utilizados na oficina.

- [`read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html#pandas-read-csv)
- [`info`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html#pandas.DataFrame.info)
- [`describe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html#pandas.DataFrame.describe)
- [`nunique`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nunique.html#pandas.DataFrame.nunique)
- [`duplicated`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html#pandas.DataFrame.duplicated)
- [`Funções built-in`](https://docs.python.org/3/library/functions.html)
- [`drop_duplicates`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html#pandas.DataFrame.drop_duplicates)
- [`dropna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html#pandas.DataFrame.dropna)
- [`unique`](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html#pandas.Series.unique)
- [`isnull`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isnull.html#pandas.DataFrame.isnull)
- [`fillna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)
- [`reset_index`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html#pandas.DataFrame.reset_index)
- [`select_dtypes`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html#pandas.DataFrame.select_dtypes)
- [`plt.rcParams`](https://matplotlib.org/stable/users/explain/customizing.html#customizing-with-dynamic-rc-settings)
- [`sns.boxplot`](https://seaborn.pydata.org/generated/seaborn.boxplot.html)
- [`replace`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html#pandas.DataFrame.replace)
- [`map`](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html#pandas.Series.map)
- [`drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html#pandas.DataFrame.drop)
- [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)
- [`concat`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)
- [`DataFrame`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
- [`corrcoef`](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html)